In [3]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import os, sys
import torch
from torch.optim import AdamW
from torchvision.io import decode_image
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
PATH_ASSETS = '~/python-workspace/CSIRO/assets'
img_dir = os.path.join(PATH_INPUT, 'train')

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [4]:
# get only columns of 'test.csv' - bare bones features
features_bb = pd.read_csv(os.path.join(PATH_ASSETS, 'test.csv')).columns.tolist()
features_bb.append('target')
df_raw = pd.read_csv(os.path.join(PATH_ASSETS, 'train.csv'))
df_raw = df_raw[features_bb]
df_raw

,sample_id,image_path,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,GDM_g,16.2750
...,...,...,...,...
1780,ID983582017__Dry_Clover_g,train/ID983582017.jpg,Dry_Clover_g,0.0000
1781,ID983582017__Dry_Dead_g,train/ID983582017.jpg,Dry_Dead_g,0.0000
1782,ID983582017__Dry_Green_g,train/ID983582017.jpg,Dry_Green_g,40.9400
1783,ID983582017__Dry_Total_g,train/ID983582017.jpg,Dry_Total_g,40.9400


In [5]:
df_raw[df_raw['image_path'].str.contains('ID1036339023.jpg')]

,sample_id,image_path,target_name,target
25,ID1036339023__Dry_Clover_g,train/ID1036339023.jpg,Dry_Clover_g,23.0755
26,ID1036339023__Dry_Dead_g,train/ID1036339023.jpg,Dry_Dead_g,2.6135
27,ID1036339023__Dry_Green_g,train/ID1036339023.jpg,Dry_Green_g,32.1910
28,ID1036339023__Dry_Total_g,train/ID1036339023.jpg,Dry_Total_g,57.8800
29,ID1036339023__GDM_g,train/ID1036339023.jpg,GDM_g,55.2665


From cell above we see that GDM and Dry_total are composites of Dry_Clover_g, Dry_Dead_g, and Dry_Green_g: Dry_Total_g is sum of green and clover, and GDM is sum of green, dead, and clover. 

Split into train and validation.

In [ ]:
y = df_raw['target']
X = df_raw.drop(['target'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, random_state = 42)
df_train = pd.concat([X_train,y_train], axis = 1)
df_val = pd.concat([X_val,y_val], axis = 1)
# For simplisity let's get rid of every grassother than clover
df_train = df_train.loc[df_train['target_name']=='Dry_Clover_g']
df_val = df_val.loc[df_val['target_name']=='Dry_Clover_g']
df_train.reset_index(inplace=True, drop=True)
df_val.reset_index(inplace=True, drop=True)

df_train

In [ ]:
# check if all images are the same size and if we need a resize pipeline right away
imgs_path = '/kaggle/input/csiro-biomass/train/'
image_list = os.listdir(imgs_path)
first_img_size = Image.open(os.path.join(imgs_path,image_list[0])).size
for img in image_list:
    if Image.open(os.path.join(imgs_path, img)).size != first_img_size:
        print(img)
#seem like we don't

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, target_transform=None):
        self.target_single = df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.target_single)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.target_single.iloc[idx, 1].split('/')[-1])
        image = decode_image(img_path)
        label = self.target_single.iloc[idx, -1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
training_data = CustomImageDataset(df_train, img_dir, transform=ToTensor())
val_data = CustomImageDataset(df_val, img_dir, transform=ToTensor())
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

In [ ]:

# # Display image and label.
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img.permute(1, 2, 0))
# plt.show()
# print(f"Label: {label}")

In [ ]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1000*2000, 512),
            nn.AdamW(),
            nn.Linear(512, 512),
            nn.AdamW(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits